## ***

In [1]:
import plotly.graph_objects as go
import os
import pandas as pd

In [2]:
ruta_de_archivos = 'C:/Users/rjaus/Downloads/bigdata/Proyecto'



ruta_de_archivos = 'C:/Users/rjaus/Downloads/bigdata/Proyecto'
archivos = os.listdir(ruta_de_archivos)

dataframes= {}

for archivo in archivos:
    if archivo.endswith('.csv'):  # Asegurarse de que solo se consideren archivos CSV
        mes = archivo.split('_')[3].split('.')[0]  # Obtener el nombre del archivo sin la extensión
        ruta_completa = os.path.join(ruta_de_archivos, archivo)  # Ruta completa al archivo
        df = pd.read_csv(ruta_completa)  # Cargar el archivo CSV en un DataFrame
        df_ventas = df.copy()

        df_ventas['Cantidad Pedida'] = pd.to_numeric(df_ventas['Cantidad Pedida'], errors='coerce')  #El errors= 'coerce' transforma los valores no numericos a NaN'
        df_ventas['Precio Unitario'] = pd.to_numeric(df_ventas['Precio Unitario'], errors='coerce')


        df_ventas = df_ventas.loc[~df_ventas['Cantidad Pedida'].isna()]
        df_ventas = df_ventas.loc[~df_ventas['Precio Unitario'].isna()]

        # Elimino todos los Nan y me quedo solo con los valores numericos
        df_ventas = df_ventas.dropna()

        # Transformo todos los valores numericos al tipo de dato int
        df_ventas['Cantidad Pedida'] = df_ventas['Cantidad Pedida'].astype(int)
        df_ventas['Precio Unitario'] = df_ventas['Precio Unitario'].astype(int)

        dataframes[mes] = df_ventas  # Agregar el DataFrame al diccionario usando el nombre del mes

meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
        'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

In [3]:
def extraer_datos(datos):
  datos['Fecha de Pedido'] = pd.to_datetime(datos['Fecha de Pedido'])
  datos['Hora'] = datos['Fecha de Pedido'].dt.hour
  datos['Dia'] = datos['Fecha de Pedido'].dt.day
  return datos


## 1. Comportamiento de las ventas en los distintos meses:


### ● ¿Cómo variaron las ventas a lo largo de los diferentes meses?


In [4]:
# Creo listas para almacenar los totales de ventas y los nombres de los meses
totales_ventas = []
nombres_meses = []

for mes in meses:
    dataframe_mes = dataframes[mes]
    dataframe_mes = extraer_datos(dataframe_mes)

    total_ventas_mes = dataframe_mes['Cantidad Pedida'].sum()
    totales_ventas.append(total_ventas_mes)
    nombres_meses.append(mes)

# Creo el objeto figura con el gráfico de barras
fig = go.Figure()

fig.add_trace(go.Bar(
    x=meses,
    y=totales_ventas,
    marker=dict(color=totales_ventas, colorscale = 'RdYlGn', colorbar=dict(title="Total de ventas", x=1.0, thickness=15)),  # Color y barra de colores
    text=totales_ventas,
    textposition='auto',  # Muestro el valor encima de las barras

))

# Configuro los ejes y los títulos
fig.update_layout(
    title="Total de ventas por mes",
    xaxis=dict(title="Mes"),
    yaxis=dict(title="Total de ventas"),
)

fig.show()

C:\Users\rjaus\AppData\Local\Temp\ipykernel_10616\3689426430.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  datos['Fecha de Pedido'] = pd.to_datetime(datos['Fecha de Pedido'])
C:\Users\rjaus\AppData\Local\Temp\ipykernel_10616\3689426430.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  datos['Fecha de Pedido'] = pd.to_datetime(datos['Fecha de Pedido'])
C:\Users\rjaus\AppData\Local\Temp\ipykernel_10616\3689426430.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  datos['Fecha de Pedido'] = pd.to_datetime(datos['Fecha de Pedido'])
C:\Users\rjaus\AppData\Local\Temp\ipy

### ● ¿Hubo algún mes que se destacó en términos de ventas?


### ● ¿Cuál es el ingreso total generado por mes?

In [5]:
ingreso_total_meses=[]
for mes in meses:
    dataframe_mes = dataframes[mes]
    ingresos_mes = dataframe_mes['Precio Unitario'].sum()
    ingreso_total_meses.append(ingresos_mes)


In [6]:
# Crea un gráfico de barras en Plotly
fig = go.Figure(data=[go.Bar(x=nombres_meses, y=ingreso_total_meses)])

# Personaliza el diseño del gráfico si es necesario
fig.update_layout(title='Ingresos Totales por Mes', xaxis_title='Meses', yaxis_title='Ingresos Totales')

# Muestra el gráfico
fig.show()

## 3. Distribución de ventas por ubicación:


### ● ¿En qué ciudades se han registrado las mayores ventas?


In [7]:
resumen_por_ciudad = pd.DataFrame(columns=["Ciudad", "Cantidad Pedida"])
for mes in dataframes:
    dataframe_mes = dataframes[mes]
    
    # Divide la columna "Dirección de Envío" en función de las comas y extrae la ciudad
    direccion_split = dataframe_mes["Dirección de Envio"].str.split(',')
    dataframe_mes["Ciudad"] = direccion_split.str[1].str.strip()
    
    # Realiza un groupby por ciudad y suma la columna "Cantidad Pedida" para el mes actual
    resumen_mes = dataframe_mes.groupby("Ciudad")["Cantidad Pedida"].sum().reset_index()
    
    resumen_por_ciudad = pd.concat([resumen_por_ciudad, resumen_mes], ignore_index=True)
    
    resumen_anual = resumen_por_ciudad.groupby("Ciudad")["Cantidad Pedida"].sum().reset_index()


In [24]:
resumen_anual["Cantidad Pedida"] = pd.to_numeric(resumen_anual["Cantidad Pedida"])
resumen_anual["Latitud"] = [33.7490, 30.2672, 42.3601, 32.7767, 34.0522, 40.7128, 45.5051, 37.7749, 47.6062]
resumen_anual["Longitud"] = [-84.3880, -97.7431, -71.0589, -96.7970, -118.2437, -74.0060, -122.6750, -122.4194, -122.3321]

fig = go.Figure()


fig.add_trace(
    go.Scattergeo(
        lon=resumen_anual['Longitud'],
        lat=resumen_anual['Latitud'],
        text=resumen_anual['Ciudad'],
        mode='markers',
        marker=dict(
            size=resumen_anual['Cantidad Pedida'] / 1000,  # Ajusta el tamaño dividiendo por 1000 para que no sea tan grande
            opacity=0.7,
            colorscale='Viridis',  # Puedes cambiar el esquema de colores según tus preferencias
            color=resumen_anual['Cantidad Pedida'],
            colorbar=dict(title='Cantidad Pedida')
        )
    )
)

fig.update_geos(
    projection_type="mercator",  # Cambia la proyección a "mercator"
    showcoastlines=True,
    coastlinecolor="Black",
    showland=True,
    landcolor="white",
)

fig.update_layout(
    title='Cantidad Pedida por Ciudades',
    geo=dict(
        scope="world",  # Ajusta el alcance del mapa a nivel mundial
        showocean=True,
        oceancolor="LightBlue",
    )
)

# Muestra el mapa
fig.show()

### ● ¿Cómo se comparan las ventas en diferentes estados o regiones?



### ● ¿Existe variación en las ventas por estado a lo largo de los meses?
